In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../src')

from plgUtil import *
from genAlgo import *
from faustGen import *

from pedalboard import Pedalboard, Plugin, Chain, Mix, Gain, Delay, Limiter, Compressor, Reverb, HighpassFilter, LowpassFilter
from pedalboard.io import AudioFile

In [5]:
# Read in a whole audio file:
with AudioFile('guitar.wav', 'r') as f:
    audio = f.read(f.frames)
    samplerate = f.samplerate

# Make a Pedalboard object, containing multiple plugins:
board = Pedalboard([Compressor(), Delay()])

# Run the audio through this pedalboard!
effected = board(audio, samplerate)

# Write the audio back as a wav file:
with AudioFile('./test2_com:del/guitar_com:del.wav', 'w', samplerate, effected.shape[0]) as f:
    f.write(effected)

In [6]:
# Read in a whole audio file:
with AudioFile('guitar.wav', 'r') as f:
    audio = f.read(f.frames)
    samplerate = f.samplerate

# Make a Pedalboard object, containing multiple plugins:
board = Pedalboard([Reverb(), LowpassFilter()])

# Run the audio through this pedalboard!
effected = board(audio, samplerate)

# Write the audio back as a wav file:
with AudioFile('./test3_rev:lpf/guitar_rev:lpf.wav', 'w', samplerate, effected.shape[0]) as f:
    f.write(effected)

In [4]:
# Read in a whole audio file:
with AudioFile('guitar.wav', 'r') as f:
    audio = f.read(f.frames)
    samplerate = f.samplerate

# Make a Pedalboard object, containing multiple plugins:
board = Pedalboard([LowpassFilter(), Mix([Gain(), HighpassFilter()]), Reverb(), Delay()])

# Run the audio through this pedalboard!
effected = board(audio, samplerate)

# Write the audio back as a wav file:
with AudioFile('./test4/guitar_lpf:gain,hpf:rev:del.wav', 'w', samplerate, effected.shape[0]) as f:
    f.write(effected)

In [2]:
from IPython.display import Audio
Audio('./guitar.wav')

In [2]:
with AudioFile('guitar.wav', 'r') as f:
    dry = f.read(f.frames)
    sr = f.samplerate
with AudioFile('./test4/guitar_lim:rev,hpf:com:gain.wav', 'r') as f:
    wet = f.read(f.frames)

In [3]:
[board, error] = evolve(dry, wet, sr)

/home/sam/anaconda3/lib/python3.9/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 2048 is greater than input length  = 2, using nperseg = 2
  warnings.warn('nperseg = {0:d} is greater than input length '


BEST MODEL THUS FAR (0)
 ERROR: 0.027868732810020447
 MODEL: <Pedalboard with 2 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Highpass cutoff_frequency_hz=50 at 0x5558ad79d280>] at 0x5558ad79c6d0>, <pedalboard.Chain with 1 plugin: [<pedalboard.Limiter threshold_db=-10 release_ms=100 at 0x5558ad16d9d0>] at 0x5558ad79d330>]>
BEST MODEL THUS FAR (1)
 ERROR: 0.023441139608621597
 MODEL: <Pedalboard with 3 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x5558ad6c5d90>] at 0x5558ad732120>, <pedalboard.Chain with 1 plugin: [<pedalboard.Highpass cutoff_frequency_hz=50 at 0x5558ad7dcce0>] at 0x5558ad7dbd20>, <pedalboard.Chain with 1 plugin: [<pedalboard.Limiter threshold_db=-10 release_ms=100 at 0x5558ad55b5e0>] at 0x5558ad5c21a0>]>
BEST MODEL THUS FAR (2)
 ERROR: 0.023441139608621597
 MODEL: <Pedalboard with 3 plugins: [<pedalboard.Chain with 1 plugin: [<pedalboard.Gain gain_db=1 at 0x5558ad6c5d90>] at 0x5558ad732120>, <pedalboard.Chain with 1 plugin: [<ped

In [9]:
write_faust_file(board)
# Read in a whole audio file:
with AudioFile('guitar.wav', 'r') as f:
    audio = f.read(f.frames)
    samplerate = f.samplerate


# Run the audio through this pedalboard!
effected = board(audio, samplerate)

# Write the audio back as a wav file:
with AudioFile('./test4/gen_lpf:gain,hpf:rev:del.wav', 'w', samplerate, effected.shape[0]) as f:
    f.write(effected)